In [1]:
%matplotlib inline

In [2]:
import QUANTAXIS as QA
import pandas as pd
import tushare as ts
import numpy as np
# import TA-Lib as ta
import scipy.signal as signal
import matplotlib.pyplot as plt

In [3]:
import tushare as ts
token = "488917d460bca13177c0b510492fc5fc04d0629bc76cc32a4f73fcf0"
ts.set_token(token)
pro = ts.pro_api()

In [4]:
import pandasql as sqldf

In [5]:
#超大板块
# 电气设备    装备制造 查看相关性高的板块
# 酿酒      46434.23
# 化工      44198.02
# 医药      40178.99
# 汽车类     34589.40
# 元器件     33136.58
supper_big_industry_code = ['880471','880446','880380','880335','880400','880390','880492']
# ['880380' '880335' '880400' '880390'] '880471','880446','880492'

# 数据准备

## 获得龙头

In [62]:
all_stock_basic_raw = pd.read_csv(r"20220903.csv",encoding='utf-8-sig')[:-1]

In [63]:
# 数据清洗
all_stock_basic_raw.loc[:,'ab_mv_raw'] = all_stock_basic_raw['AB股总市值'].map(lambda x: x.strip().replace('亿',''))
all_stock_basic = all_stock_basic_raw[all_stock_basic_raw['ab_mv_raw'] != '--']
# 数据处理
all_stock_basic.loc[:,'parent_industry'] = all_stock_basic['所属行业'].map(lambda x: x.strip().split('-')[0])
all_stock_basic.loc[:,'child_industry'] = all_stock_basic['所属行业'].map(lambda x: x.strip().split('-')[1])
all_stock_basic.loc[:,('ab_mv')] = all_stock_basic['ab_mv_raw'].map(lambda x: pd.to_numeric(x))

In [64]:
# 数据分析
blk_mv = pd.DataFrame(all_stock_basic.groupby(['细分行业'])['ab_mv'].agg(['sum','count'])).rename(columns={'sum':'blk_mv'})
all_stock_basic_1 = all_stock_basic.merge(blk_mv,left_on = '细分行业',right_index=True)
all_stock_basic_1.loc[:,'mv_share'] = all_stock_basic_1['ab_mv']/all_stock_basic_1['blk_mv']*100

In [69]:
# 龙头
top1_stocks_all = all_stock_basic_1.groupby(['细分行业'])\
    .apply(lambda x:x.sort_values(by='mv_share',ascending=False).head(1))\
    .filter(['代码','名称','ab_mv','blk_mv','mv_share','count']).sort_values(by='count',ascending=False)

### 非小型板块龙头

In [71]:
# 筛选非小板块
top1_stocks_non_small = top1_stocks_all.query('blk_mv>3000 and count>4').droplevel(level=1)
top1_stocks_non_small

,代码,名称,ab_mv,blk_mv,mv_share,count
细分行业,,,,,,
软件服务,002230,科大讯飞,868.85,22044.60,3.941328,273
电气设备,300750,宁德时代,11378.94,62863.98,18.100890,263
元器件,002475,立讯精密,2685.39,33136.58,8.104005,252
化工原料,600309,万华化学,2730.01,29082.09,9.387255,226
专用机械,300316,晶盛机电,969.49,17882.27,5.421515,217
...,...,...,...,...,...,...
石油加工,600028,中国石化,4070.76,6130.75,66.399054,11
电信运营,601728,中国电信,2942.17,5706.43,51.558856,11
空运,601111,中国国航,975.29,4127.72,23.627814,9


### 小型板块龙头

In [76]:
top1_stocks_small_blk  = top1_stocks_all.query('blk_mv < 3000').droplevel(level=1)
top1_stocks_small_blk

,代码,名称,ab_mv,blk_mv,mv_share,count
细分行业,,,,,,
文教休闲,603899,晨光股份,407.85,2620.09,15.566259,45
纺织,603055,台华新材,92.61,1766.41,5.242837,44
百货,600655,豫园股份,312.84,2491.49,12.556342,38
其他建材,002271,东方雨虹,768.78,2957.19,25.996977,33
装修装饰,603098,森特股份,166.76,1257.84,13.257648,31
造纸,002078,太阳纸业,319.75,2083.78,15.344710,31
出版业,601928,凤凰传媒,228.53,2095.71,10.904658,28
染料涂料,603737,三棵树,347.75,1967.16,17.677769,28
综合类,002967,广电计量,127.93,1356.61,9.430124,27


## 获取所有细分行业数据

In [82]:
detail_industry = pd.read_csv('申万细分行业.csv')

In [83]:
detail_industry.loc[:,('stock_code')] = detail_industry['代码'].astype(str)

In [85]:
# 转换code 为str
detail_industry_code = detail_industry['stock_code'].to_list()

In [86]:
#获取板块数据
detail_industry_data = QA.QA_fetch_index_day_adv(detail_industry_code,'2022-01-01','2022-09-08')

## 获取主要指数数据

In [ ]:
index_code_name_map = pd.DataFrame(['上证指数','深证综指','上证50','中证500','创业板指','科创50'],\
                                   index = ['000001','399106','000016','000905','399006','000688'],columns=['名称'])

In [87]:
# 获取主要指数数据
main_index_data = QA.QA_fetch_index_day_adv(['000001','399106','000016','000905','399006','000688'],'2022-01-01','2022-09-08')

# 细分行业分析

In [88]:
detail_industry

,代码,名称,stock_code
0,880313,石油贸易,880313
1,880363,林业,880363
2,880431,船舶,880431
3,880490,通信设备,880490
4,880444,农用机械,880444
...,...,...,...
105,880302,煤炭开采,880302
106,880373,乳制品,880373
107,880351,矿物制品,880351
108,880382,啤酒,880382


In [89]:
detail_industry_code_name_map = detail_industry.drop(['代码'],axis=1).set_index('stock_code')

In [90]:
detail_industry_code_name_map

,名称
stock_code,
880313,石油贸易
880363,林业
880431,船舶
880490,通信设备
880444,农用机械
...,...
880302,煤炭开采
880373,乳制品
880351,矿物制品


In [91]:
#获取细分行业的板块名
index_detail_industry_code_name_map = pd.concat([index_code_name_map,detail_industry_code_name_map])

In [92]:
# super_big_industry_data = QA.QA_fetch_index_day_adv(supper_big_industry_code,'2022-01-01','2022-09-02')

In [93]:
# supper_and_detail_industry = pd.concat([super_big_industry_data.pivot('close'),detail_industry_data_new],axis=1)

In [94]:
# supper_and_detail_industry_corr = supper_and_detail_industry.pct_change()
# detail_industry_corr = detail_industry_data.pivot('close').pct_change().corr()
# 获得主要指数和板块的相关性
detail_industry_corr = pd.concat([detail_industry_data.pivot('close').pct_change(),main_index_data.pivot('close').pct_change()],axis=1)\
                            .corr()

In [18]:
# 对板块进行分类
blk_category = pd.read_csv('blk_category.csv').drop('Unnamed: 0',axis=1).set_index('细分行业')

In [19]:
blk_category.set_index

<bound method DataFrame.set_index of      category
细分行业         
银行       超大板块
电气设备     超大板块
白酒       超大板块
元器件      超大板块
化工原料     超大板块
...       ...
电器连锁     小型板块
公共交通     小型板块
渔业       小型板块
林业       小型板块
公路       小型板块

[110 rows x 1 columns]>

## 主要指数相关性分析

### 上证指数

In [20]:
# 上证
one_blk_corr = detail_industry_corr['000001'].filter(regex='880*',axis=0)
one_blk_related_blk = one_blk_corr[one_blk_corr>0.7].sort_values(ascending= False)
上证 = one_blk_related_blk.to_frame().join(index_detail_industry_code_name_map).merge(blk_category,left_on='名称',right_index=True)
上证.insert(1,'index','000001')
上证

,000001,index,名称,category
code,,,,
880432,0.882768,000001,运输设备,中小型板块
880456,0.832876,000001,环境保护,中型板块
880336,0.821197,000001,化工原料,超大板块
880490,0.818953,000001,通信设备,大板块
880421,0.817902,000001,广告包装,中小型板块
880411,0.812458,000001,其他商业,小型板块
880447,0.811962,000001,工程机械,中型板块
880414,0.810664,000001,商贸代理,小型板块
880474,0.806899,000001,多元金融,中小型板块


### 深证综指

In [21]:
# 深证综指
one_blk_corr = detail_industry_corr['399106'].filter(regex='880*',axis=0)
one_blk_related_blk = one_blk_corr[one_blk_corr>0.7].sort_values(ascending= False)
深证综指 = one_blk_related_blk.to_frame().join(index_detail_industry_code_name_map).merge(blk_category,left_on='名称',right_index=True)
深证综指.insert(1,'index','399106')
深证综指

,399106,index,名称,category
code,,,,
880490,0.908160,399106,通信设备,大板块
880492,0.901729,399106,元器件,超大板块
880445,0.892743,399106,专用机械,大板块
880336,0.885351,399106,化工原料,超大板块
880432,0.878245,399106,运输设备,中小型板块
880493,0.870893,399106,软件服务,超大板块
880456,0.866650,399106,环境保护,中型板块
880421,0.862718,399106,广告包装,中小型板块
880448,0.859887,399106,电器仪表,中型板块


### 上证50

In [22]:
# 上证50
one_blk_corr = detail_industry_corr['000016'].filter(regex='880*',axis=0)
one_blk_related_blk = one_blk_corr[one_blk_corr>0.7].sort_values(ascending= False)
上证50 = one_blk_related_blk.to_frame().join(index_detail_industry_code_name_map).merge(blk_category,left_on='名称',right_index=True)
上证50.insert(1,'index','000016')
上证50

,000016,index,名称,category
code,,,,
880381,0.808004,000016,白酒,超大板块
880387,0.777413,000016,家用电器,大板块
880373,0.758366,000016,乳制品,中小型板块
880472,0.757821,000016,证券,超大板块
880473,0.757480,000016,保险,大板块
880471,0.740435,000016,银行,超大板块
880375,0.703784,000016,食品,大板块
880374,0.701669,000016,软饮料,小型板块


### 中证500

In [23]:
# 中证500
one_blk_corr = detail_industry_corr['000905'].filter(regex='880*',axis=0)
one_blk_related_blk = one_blk_corr[one_blk_corr>0.8].sort_values(ascending= False)
中证500 = one_blk_related_blk.to_frame().join(index_detail_industry_code_name_map).merge(blk_category,left_on='名称',right_index=True)
中证500.insert(1,'index','000905')
中证500

,000905,index,名称,category
code,,,,
880432,0.900652,000905,运输设备,中小型板块
880490,0.885121,000905,通信设备,大板块
880336,0.878260,000905,化工原料,超大板块
880456,0.875080,000905,环境保护,中型板块
880414,0.865142,000905,商贸代理,小型板块
880320,0.857878,000905,特种钢,中小型板块
880492,0.848695,000905,元器件,超大板块
880411,0.848208,000905,其他商业,小型板块
880421,0.848184,000905,广告包装,中小型板块


In [24]:
# 创业板指
one_blk_corr = detail_industry_corr['399006'].filter(regex='880*',axis=0)
one_blk_related_blk = one_blk_corr[one_blk_corr>0.7].sort_values(ascending= False)
创业板指 = one_blk_related_blk.to_frame().join(index_detail_industry_code_name_map).merge(blk_category,left_on='名称',right_index=True)
创业板指.insert(1,'index','399006')
创业板指

,399006,index,名称,category
code,,,,
880446,0.913331,399006,电气设备,超大板块
880492,0.857650,399006,元器件,超大板块
880445,0.831031,399006,专用机械,大板块
880448,0.806591,399006,电器仪表,中型板块
880338,0.801482,399006,塑料,中小型板块
880490,0.797974,399006,通信设备,大板块
880336,0.796544,399006,化工原料,超大板块
880392,0.768362,399006,汽车配件,大板块
880351,0.761658,399006,矿物制品,中小型板块


In [25]:
# 科创50
one_blk_corr = detail_industry_corr['000688'].filter(regex='880*',axis=0)
one_blk_related_blk = one_blk_corr[one_blk_corr>0.7].sort_values(ascending= False)
科创50 = one_blk_related_blk.to_frame().join(index_detail_industry_code_name_map).merge(blk_category,left_on='名称',right_index=True)
科创50.insert(1,'index','000688')
科创50

,000688,index,名称,category
code,,,,
880492,0.899654,000688,元器件,超大板块
880491,0.878607,000688,半导体,超大板块
880445,0.855162,000688,专用机械,大板块
880490,0.841989,000688,通信设备,大板块
880338,0.826524,000688,塑料,中小型板块
880446,0.826024,000688,电气设备,超大板块
880448,0.765854,000688,电器仪表,中型板块
880336,0.757132,000688,化工原料,超大板块
880392,0.751398,000688,汽车配件,大板块


## 板块分析

### 银行

In [105]:
bank_blk_code = '880471'

In [26]:
# 银行
one_blk_corr = detail_industry_corr['880471']
one_blk_related_blk = one_blk_corr[one_blk_corr>0.6].sort_values(ascending= False)
银行 = one_blk_related_blk.to_frame().join(detail_industry_code_name_map).merge(blk_category,left_on='名称',right_index=True)
银行.insert(1,'index','880471')
银行

,880471,index,名称,category
code,,,,
880471,1.000000,880471,银行,超大板块
880473,0.785188,880471,保险,大板块
880483,0.679759,880471,全国地产,中型板块
880460,0.659910,880471,铁路,中小型板块
880346,0.654351,880471,水泥,中小型板块
880312,0.618992,880471,石油加工,中型板块
880468,0.618866,880471,港口,中小型板块
880477,0.616367,880471,建筑工程,大板块
880319,0.615836,880471,普钢,中型板块


In [81]:
top1_stocks_non_small.loc['银行']

代码                601398
名称                  工商银行
ab_mv       11728.130000
blk_mv      64237.230000
mv_share       18.257528
count                 42
Name: 银行, dtype: object

In [125]:
# top1_stocks_non_small.loc['银行']
top1 = top1_stocks_non_small.loc['银行']['代码']
top1_data = QA.QA_fetch_stock_day_adv(top1,'2022-01-01','2022-09-08').pivot('close').pct_change()
pd.concat([detail_industry_data.pivot('close')[[bank_blk_code]].pct_change(),top1_data],axis=1).corr()

code,880471,601398
code,,
880471,1.000000,0.629776
601398,0.629776,1.000000


### 电气设备

In [27]:
# 电气设备
one_blk_corr = detail_industry_corr['880446']
one_blk_related_blk = one_blk_corr[one_blk_corr>0.7].sort_values(ascending= False)
电气设备 = one_blk_related_blk.to_frame().join(detail_industry_code_name_map).merge(blk_category,left_on='名称',right_index=True)
电气设备.insert(1,'index','880446')
电气设备

,880446,index,名称,category
code,,,,
880446,1.000000,880446,电气设备,超大板块
880445,0.889178,880446,专用机械,大板块
880338,0.874395,880446,塑料,中小型板块
880351,0.844962,880446,矿物制品,中小型板块
880439,0.819630,880446,机械基件,中型板块
880336,0.819373,880446,化工原料,超大板块
880492,0.814755,880446,元器件,超大板块
880392,0.803896,880446,汽车配件,大板块
880347,0.776505,880446,玻璃,中小型板块


In [126]:
top1_stocks_non_small.loc['电气设备']

代码               300750
名称                 宁德时代
ab_mv       11378.94000
blk_mv      62863.98000
mv_share       18.10089
count               263
Name: 电气设备, dtype: object

In [127]:
# top1_stocks_non_small.loc['电气设备']
top1 = top1_stocks_non_small.loc['电气设备']['代码']
top1_data = QA.QA_fetch_stock_day_adv(top1,'2022-01-01','2022-09-08').pivot('close').pct_change()
pd.concat([detail_industry_data.pivot('close')[['880446']].pct_change(),top1_data],axis=1).corr()

code,880446,300750
code,,
880446,1.000000,0.819629
300750,0.819629,1.000000


### 白酒

In [28]:
# 白酒
one_blk_corr = detail_industry_corr['880381']
one_blk_related_blk = one_blk_corr[one_blk_corr>0.6].sort_values(ascending= False)
白酒 = one_blk_related_blk.to_frame().join(detail_industry_code_name_map).merge(blk_category,left_on='名称',right_index=True)
白酒.insert(1,'index','880381')
白酒

,880381,index,名称,category
code,,,,
880381,1.000000,880381,白酒,超大板块
880382,0.774453,880381,啤酒,小型板块
880425,0.702841,880381,旅游服务,中小型板块
880383,0.676759,880381,红黄酒,小型板块
880375,0.668494,880381,食品,大板块
880373,0.654608,880381,乳制品,中小型板块
880387,0.634762,880381,家用电器,大板块
880355,0.605385,880381,日用化工,小型板块


In [128]:
top1_stocks_non_small.loc['白酒']

代码                600519
名称                  贵州茅台
ab_mv       23553.710000
blk_mv      43895.070000
mv_share       53.659124
count                 20
Name: 白酒, dtype: object

In [129]:
# top1_stocks_non_small.loc['电气设备']
top1 = top1_stocks_non_small.loc['白酒']['代码']
top1_data = QA.QA_fetch_stock_day_adv(top1,'2022-01-01','2022-09-08').pivot('close').pct_change()
pd.concat([detail_industry_data.pivot('close')[['880381']].pct_change(),top1_data],axis=1).corr()

code,880381,600519
code,,
880381,1.000000,0.942528
600519,0.942528,1.000000


### 元器件

In [29]:
# 元器件
one_blk_corr = detail_industry_corr['880492']
one_blk_related_blk = one_blk_corr[one_blk_corr>0.75].sort_values(ascending= False)
元器件 = one_blk_related_blk.to_frame().join(detail_industry_code_name_map).merge(blk_category,left_on='名称',right_index=True)
元器件.insert(1,'index','880492')
元器件

,880492,index,名称,category
code,,,,
880492,1.000000,880492,元器件,超大板块
880490,0.917303,880492,通信设备,大板块
880445,0.888856,880492,专用机械,大板块
880491,0.860644,880492,半导体,超大板块
880448,0.847799,880492,电器仪表,中型板块
880338,0.825424,880492,塑料,中小型板块
880493,0.820409,880492,软件服务,超大板块
880446,0.814755,880492,电气设备,超大板块
880439,0.813457,880492,机械基件,中型板块


In [130]:
top1_stocks_non_small.loc['元器件']

代码                002475
名称                  立讯精密
ab_mv        2685.390000
blk_mv      33136.580000
mv_share        8.104005
count                252
Name: 元器件, dtype: object

In [131]:
# top1_stocks_non_small.loc['电气设备']
top1 = top1_stocks_non_small.loc['元器件']['代码']
top1_data = QA.QA_fetch_stock_day_adv(top1,'2022-01-01','2022-09-08').pivot('close').pct_change()
pd.concat([detail_industry_data.pivot('close')[['880492']].pct_change(),top1_data],axis=1).corr()

code,880492,002475
code,,
880492,1.000000,0.746748
002475,0.746748,1.000000


### 证券

In [30]:
# 证券
one_blk_corr = detail_industry_corr['880472']
one_blk_related_blk = one_blk_corr[one_blk_corr>0.7].sort_values(ascending= False)
证券 = one_blk_related_blk.to_frame().join(detail_industry_code_name_map).merge(blk_category,left_on='名称',right_index=True)
证券.insert(1,'index','880472')
证券

,880472,index,名称,category
code,,,,
880472,1.000000,880472,证券,超大板块
880474,0.819134,880472,多元金融,中小型板块
880473,0.776774,880472,保险,大板块


In [132]:
top1_stocks_non_small.loc['证券']

代码               300059
名称                 东方财富
ab_mv        2893.90000
blk_mv      26627.42000
mv_share       10.86812
count                49
Name: 证券, dtype: object

In [137]:
# top1_stocks_non_small.loc['电气设备']
top1 = top1_stocks_non_small.loc['证券']['代码']
top1_data = QA.QA_fetch_stock_day_adv(top1,'2022-01-01','2022-09-08').pivot('close').pct_change()
pd.concat([detail_industry_data.pivot('close')[['880472']].pct_change(),top1_data],axis=1).corr()

code,880472,300059
code,,
880472,1.000000,0.740722
300059,0.740722,1.000000


### 化工原料

In [32]:
# 化工原料
one_blk_corr = detail_industry_corr['880336']
one_blk_related_blk = one_blk_corr[one_blk_corr>0.75].sort_values(ascending= False)
化工原料 = one_blk_related_blk.to_frame().join(detail_industry_code_name_map).merge(blk_category,left_on='名称',right_index=True)
化工原料.insert(1,'index','880336')
化工原料

,880336,index,名称,category
code,,,,
880336,1.000000,880336,化工原料,超大板块
880329,0.881156,880336,小金属,大板块
880445,0.853613,880336,专用机械,大板块
880338,0.847188,880336,塑料,中小型板块
880351,0.822563,880336,矿物制品,中小型板块
880320,0.820841,880336,特种钢,中小型板块
880446,0.819373,880336,电气设备,超大板块
880337,0.817669,880336,农药化肥,中型板块
880347,0.816649,880336,玻璃,中小型板块


In [138]:
top1_stocks_non_small.loc['化工原料']

代码                600309
名称                  万华化学
ab_mv        2730.010000
blk_mv      29082.090000
mv_share        9.387255
count                226
Name: 化工原料, dtype: object

In [139]:
# top1_stocks_non_small.loc['化工原料']
top1 = top1_stocks_non_small.loc['化工原料']['代码']
top1_data = QA.QA_fetch_stock_day_adv(top1,'2022-01-01','2022-09-08').pivot('close').pct_change()
pd.concat([detail_industry_data.pivot('close')[['880336']].pct_change(),top1_data],axis=1).corr()

code,880336,600309
code,,
880336,1.000000,0.463747
600309,0.463747,1.000000


In [140]:
# 老大和板块相关性不大

### 半导体

In [33]:
# 半导体
one_blk_corr = detail_industry_corr['880491']
one_blk_related_blk = one_blk_corr[one_blk_corr>0.75].sort_values(ascending= False)
半导体 = one_blk_related_blk.to_frame().join(detail_industry_code_name_map).merge(blk_category,left_on='名称',right_index=True)
半导体.insert(1,'index','880491')
半导体

,880491,index,名称,category
code,,,,
880491,1.000000,880491,半导体,超大板块
880492,0.860644,880491,元器件,超大板块
880490,0.797509,880491,通信设备,大板块


In [141]:
top1_stocks_non_small.loc['半导体']

代码                002371
名称                  北方华创
ab_mv        1505.860000
blk_mv      27098.160000
mv_share        5.557056
count                124
Name: 半导体, dtype: object

In [142]:
# top1_stocks_non_small.loc['化工原料']
top1 = top1_stocks_non_small.loc['半导体']['代码']
top1_data = QA.QA_fetch_stock_day_adv(top1,'2022-01-01','2022-09-08').pivot('close').pct_change()
pd.concat([detail_industry_data.pivot('close')[['880491']].pct_change(),top1_data],axis=1).corr()

code,880491,002371
code,,
880491,1.000000,0.846177
002371,0.846177,1.000000


### 软件服务

In [35]:
# 软件服务
one_blk_corr = detail_industry_corr['880493']
one_blk_related_blk = one_blk_corr[one_blk_corr>0.8].sort_values(ascending= False)
软件服务 = one_blk_related_blk.to_frame().join(detail_industry_code_name_map).merge(blk_category,left_on='名称',right_index=True)
软件服务.insert(1,'index','880493')
软件服务

,880493,index,名称,category
code,,,,
880493,1.000000,880493,软件服务,超大板块
880489,0.940013,880493,IT设备,中小型板块
880490,0.877155,880493,通信设备,大板块
880494,0.865020,880493,互联网,中型板块
880421,0.827916,880493,广告包装,中小型板块
880492,0.820409,880493,元器件,超大板块


In [143]:
top1_stocks_non_small.loc['软件服务']

代码                002230
名称                  科大讯飞
ab_mv         868.850000
blk_mv      22044.600000
mv_share        3.941328
count                273
Name: 软件服务, dtype: object

In [144]:
# top1_stocks_non_small.loc['化工原料']
top1 = top1_stocks_non_small.loc['软件服务']['代码']
top1_data = QA.QA_fetch_stock_day_adv(top1,'2022-01-01','2022-09-08').pivot('close').pct_change()
pd.concat([detail_industry_data.pivot('close')[['880493']].pct_change(),top1_data],axis=1).corr()

code,880493,002230
code,,
880493,1.000000,0.730379
002230,0.730379,1.000000


### 医疗保健

In [37]:
# 医疗保健
one_blk_corr = detail_industry_corr['880398']
one_blk_related_blk = one_blk_corr[one_blk_corr>0.75].sort_values(ascending= False)
医疗保健 = one_blk_related_blk.to_frame().join(detail_industry_code_name_map).merge(blk_category,left_on='名称',right_index=True)
医疗保健.insert(1,'index','880398')
医疗保健

,880398,index,名称,category
code,,,,
880398,1.000000,880398,医疗保健,超大板块
880401,0.880280,880398,化学制药,大板块
880402,0.845908,880398,生物制药,大板块
880403,0.767502,880398,中成药,中型板块


In [ ]:
top1_stocks_non_small.loc['医疗保健']

In [145]:
# top1_stocks_non_small.loc['化工原料']
top1 = top1_stocks_non_small.loc['医疗保健']['代码']
top1_data = QA.QA_fetch_stock_day_adv(top1,'2022-01-01','2022-09-08').pivot('close').pct_change()
pd.concat([detail_industry_data.pivot('close')[['880398']].pct_change(),top1_data],axis=1).corr()

code,880398,300760
code,,
880398,1.000000,0.744716
300760,0.744716,1.000000


### 化学制药

In [38]:
# 化学制药
one_blk_corr = detail_industry_corr['880401']
one_blk_related_blk = one_blk_corr[one_blk_corr>0.8].sort_values(ascending= False)
化学制药 = one_blk_related_blk.to_frame().join(detail_industry_code_name_map).merge(blk_category,left_on='名称',right_index=True)
化学制药.insert(1,'index','880401')
化学制药

,880401,index,名称,category
code,,,,
880401,1.000000,880401,化学制药,大板块
880402,0.897752,880401,生物制药,大板块
880398,0.880280,880401,医疗保健,超大板块
880403,0.813067,880401,中成药,中型板块


In [147]:
top1_stocks_non_small.loc['化学制药']

代码                603259
名称                  药明康德
ab_mv        2212.370000
blk_mv      18012.010000
mv_share       12.282749
count                136
Name: 化学制药, dtype: object

In [148]:
# top1_stocks_non_small.loc['化工原料']
top1 = top1_stocks_non_small.loc['化学制药']['代码']
top1_data = QA.QA_fetch_stock_day_adv(top1,'2022-01-01','2022-09-08').pivot('close').pct_change()
pd.concat([detail_industry_data.pivot('close')[['880401']].pct_change(),top1_data],axis=1).corr()

code,880401,603259
code,,
880401,1.000000,0.698586
603259,0.698586,1.000000


### 煤炭开采

In [39]:
# 煤炭开采
one_blk_corr = detail_industry_corr['880302']
one_blk_related_blk = one_blk_corr[one_blk_corr>0.65].sort_values(ascending= False)
煤炭开采 = one_blk_related_blk.to_frame().join(detail_industry_code_name_map).merge(blk_category,left_on='名称',right_index=True)
煤炭开采.insert(1,'index','880302')
煤炭开采

,880302,index,名称,category
code,,,,
880302,1.000000,880302,煤炭开采,大板块
880311,0.660373,880302,石油开采,中型板块


In [149]:
top1_stocks_non_small.loc['煤炭开采']

代码               601088
名称                 中国神华
ab_mv        5056.15000
blk_mv      14325.53000
mv_share       35.29468
count                26
Name: 煤炭开采, dtype: object

In [150]:
# top1_stocks_non_small.loc['化工原料']
top1 = top1_stocks_non_small.loc['煤炭开采']['代码']
top1_data = QA.QA_fetch_stock_day_adv(top1,'2022-01-01','2022-09-08').pivot('close').pct_change()
pd.concat([detail_industry_data.pivot('close')[['880302']].pct_change(),top1_data],axis=1).corr()

code,880302,601088
code,,
880302,1.000000,0.905583
601088,0.905583,1.000000


### 汽车配件

In [152]:
# 汽车配件
one_blk_corr = detail_industry_corr['880392']
one_blk_related_blk = one_blk_corr[one_blk_corr>0.75].sort_values(ascending= False)
汽车配件 = one_blk_related_blk.to_frame().join(detail_industry_code_name_map).merge(blk_category,left_on='名称',right_index=True)
汽车配件.insert(1,'index','880392')
汽车配件

,880392,index,名称,category
code,,,,
880392,1.000000,880392,汽车配件,大板块
880445,0.870529,880392,专用机械,大板块
880439,0.828076,880392,机械基件,中型板块
880391,0.822812,880392,汽车整车,大板块
880492,0.809787,880392,元器件,超大板块
880338,0.808349,880392,塑料,中小型板块
880446,0.803896,880392,电气设备,超大板块
880351,0.789260,880392,矿物制品,中小型板块
880490,0.775501,880392,通信设备,大板块


In [151]:
top1_stocks_non_small.loc['汽车配件']

代码                601689
名称                  拓普集团
ab_mv         883.510000
blk_mv      17607.840000
mv_share        5.017708
count                200
Name: 汽车配件, dtype: object

In [153]:
# top1_stocks_non_small.loc['化工原料']
top1 = top1_stocks_non_small.loc['汽车配件']['代码']
top1_data = QA.QA_fetch_stock_day_adv(top1,'2022-01-01','2022-09-08').pivot('close').pct_change()
pd.concat([detail_industry_data.pivot('close')[['880392']].pct_change(),top1_data],axis=1).corr()

code,880392,601689
code,,
880392,1.000000,0.740936
601689,0.740936,1.000000


### 家用电器

In [40]:
# 家用电器
one_blk_corr = detail_industry_corr['880387']
one_blk_related_blk = one_blk_corr[one_blk_corr>0.7].sort_values(ascending= False)
家用电器 = one_blk_related_blk.to_frame().join(detail_industry_code_name_map).merge(blk_category,left_on='名称',right_index=True)
家用电器.insert(1,'index','880387')
家用电器

,880387,index,名称,category
code,,,,
880387,1.000000,880387,家用电器,大板块
880399,0.767237,880387,家居用品,中小型板块
880340,0.721512,880387,染料涂料,小型板块
880447,0.720888,880387,工程机械,中型板块
880432,0.714312,880387,运输设备,中小型板块
880374,0.710831,880387,软饮料,小型板块


In [154]:
top1_stocks_non_small.loc['家用电器']

代码                000333
名称                  美的集团
ab_mv        3754.440000
blk_mv      16237.720000
mv_share       23.121719
count                 79
Name: 家用电器, dtype: object

In [155]:
# top1_stocks_non_small.loc['化工原料']
top1 = top1_stocks_non_small.loc['家用电器']['代码']
top1_data = QA.QA_fetch_stock_day_adv(top1,'2022-01-01','2022-09-08').pivot('close').pct_change()
pd.concat([detail_industry_data.pivot('close')[['880387']].pct_change(),top1_data],axis=1).corr()

code,880387,000333
code,,
880387,1.000000,0.878319
000333,0.878319,1.000000


### 专用机械

In [42]:
# 专用机械
one_blk_corr = detail_industry_corr['880445']
one_blk_related_blk = one_blk_corr[one_blk_corr>0.75].sort_values(ascending= False)
专用机械 = one_blk_related_blk.to_frame().join(detail_industry_code_name_map).merge(blk_category,left_on='名称',right_index=True)
专用机械.insert(1,'index','880445')
专用机械

,880445,index,名称,category
code,,,,
880445,1.000000,880445,专用机械,大板块
880439,0.934573,880445,机械基件,中型板块
880446,0.889178,880445,电气设备,超大板块
880492,0.888856,880445,元器件,超大板块
880338,0.886384,880445,塑料,中小型板块
880392,0.870529,880445,汽车配件,大板块
880351,0.869947,880445,矿物制品,中小型板块
880336,0.853613,880445,化工原料,超大板块
880490,0.850322,880445,通信设备,大板块


In [156]:
top1_stocks_non_small.loc['专用机械']

代码                300316
名称                  晶盛机电
ab_mv         969.490000
blk_mv      17882.270000
mv_share        5.421515
count                217
Name: 专用机械, dtype: object

In [157]:
# top1_stocks_non_small.loc['化工原料']
top1 = top1_stocks_non_small.loc['专用机械']['代码']
top1_data = QA.QA_fetch_stock_day_adv(top1,'2022-01-01','2022-09-08').pivot('close').pct_change()
pd.concat([detail_industry_data.pivot('close')[['880445']].pct_change(),top1_data],axis=1).corr()

code,880445,300316
code,,
880445,1.000000,0.742251
300316,0.742251,1.000000


### 建筑工程

In [44]:
# 建筑工程
one_blk_corr = detail_industry_corr['880477']
one_blk_related_blk = one_blk_corr[one_blk_corr>0.7].sort_values(ascending= False)
建筑工程 = one_blk_related_blk.to_frame().join(detail_industry_code_name_map).merge(blk_category,left_on='名称',right_index=True)
建筑工程.insert(1,'index','880477')
建筑工程

,880477,index,名称,category
code,,,,
880477,1.000000,880477,建筑工程,大板块
880346,0.834117,880477,水泥,中小型板块
880484,0.745531,880477,区域地产,中型板块
880319,0.738937,880477,普钢,中型板块
880485,0.731122,880477,园区开发,小型板块
880321,0.715120,880477,钢加工,小型板块


In [ ]:
top1_stocks_non_small.loc['建筑工程']

In [158]:
# top1_stocks_non_small.loc['化工原料']
top1 = top1_stocks_non_small.loc['建筑工程']['代码']
top1_data = QA.QA_fetch_stock_day_adv(top1,'2022-01-01','2022-09-08').pivot('close').pct_change()
pd.concat([detail_industry_data.pivot('close')[['880477']].pct_change(),top1_data],axis=1).corr()

code,880477,601668
code,,
880477,1.000000,0.828249
601668,0.828249,1.000000


### 通信设备

In [45]:
# 通信设备
one_blk_corr = detail_industry_corr['880490']
one_blk_related_blk = one_blk_corr[one_blk_corr>0.7].sort_values(ascending= False)
通信设备 = one_blk_related_blk.to_frame().join(detail_industry_code_name_map).merge(blk_category,left_on='名称',right_index=True)
通信设备.insert(1,'index','880490')
通信设备

,880490,index,名称,category
code,,,,
880490,1.000000,880490,通信设备,大板块
880492,0.917303,880490,元器件,超大板块
880493,0.877155,880490,软件服务,超大板块
880489,0.871767,880490,IT设备,中小型板块
880445,0.850322,880490,专用机械,大板块
880448,0.844905,880490,电器仪表,中型板块
880432,0.831022,880490,运输设备,中小型板块
880439,0.823497,880490,机械基件,中型板块
880421,0.806867,880490,广告包装,中小型板块


In [159]:
top1_stocks_non_small.loc['通信设备']

代码                601138
名称                  工业富联
ab_mv        1849.060000
blk_mv      14625.450000
mv_share       12.642756
count                138
Name: 通信设备, dtype: object

In [160]:
# top1_stocks_non_small.loc['化工原料']
top1 = top1_stocks_non_small.loc['通信设备']['代码']
top1_data = QA.QA_fetch_stock_day_adv(top1,'2022-01-01','2022-09-08').pivot('close').pct_change()
pd.concat([detail_industry_data.pivot('close')[['880490']].pct_change(),top1_data],axis=1).corr()

code,880490,601138
code,,
880490,1.000000,0.660657
601138,0.660657,1.000000


### 小金属	

In [47]:
# 小金属
one_blk_corr = detail_industry_corr['880329']
one_blk_related_blk = one_blk_corr[one_blk_corr>0.7].sort_values(ascending= False)
小金属	 = one_blk_related_blk.to_frame().join(detail_industry_code_name_map).merge(blk_category,left_on='名称',right_index=True)
小金属	.insert(1,'index','880329')
小金属	

,880329,index,名称,category
code,,,,
880329,1.000000,880329,小金属,大板块
880336,0.881156,880329,化工原料,超大板块
880351,0.799480,880329,矿物制品,中小型板块
880320,0.785380,880329,特种钢,中小型板块
880445,0.778919,880329,专用机械,大板块
880337,0.769133,880329,农药化肥,中型板块
880446,0.760065,880329,电气设备,超大板块
880327,0.759206,880329,铅锌,小型板块
880338,0.740007,880329,塑料,中小型板块


In [161]:
top1_stocks_non_small.loc['小金属']

代码                002466
名称                  天齐锂业
ab_mv        1554.200000
blk_mv      13060.150000
mv_share       11.900323
count                 53
Name: 小金属, dtype: object

In [162]:
# top1_stocks_non_small.loc['化工原料']
top1 = top1_stocks_non_small.loc['小金属']['代码']
top1_data = QA.QA_fetch_stock_day_adv(top1,'2022-01-01','2022-09-08').pivot('close').pct_change()
pd.concat([detail_industry_data.pivot('close')[['880329']].pct_change(),top1_data],axis=1).corr()

code,880329,002466
code,,
880329,1.000000,0.816098
002466,0.816098,1.000000


### 石油开采

In [165]:
# 石油开采
one_blk_corr = detail_industry_corr['880311']
one_blk_related_blk = one_blk_corr[one_blk_corr>0.75].sort_values(ascending= False)
石油开采 = one_blk_related_blk.to_frame().join(detail_industry_code_name_map).merge(blk_category,left_on='名称',right_index=True)
石油开采.insert(1,'index','880311')
石油开采

,880311,index,名称,category
code,,,,
880311,1.000000,880311,石油开采,中型板块
880312,0.825442,880311,石油加工,中型板块


In [163]:
top1_stocks_non_small.loc['石油开采']

代码                601857
名称                  中国石油
ab_mv        8776.180000
blk_mv      12025.270000
mv_share       72.981147
count                 20
Name: 石油开采, dtype: object

In [166]:
# top1_stocks_non_small.loc['化工原料']
top1 = top1_stocks_non_small.loc['石油开采']['代码']
top1_data = QA.QA_fetch_stock_day_adv(top1,'2022-01-01','2022-09-08').pivot('close').pct_change()
pd.concat([detail_industry_data.pivot('close')[['880311']].pct_change(),top1_data],axis=1).corr()

code,880311,601857
code,,
880311,1.000000,0.984796
601857,0.984796,1.000000


### 汽车整车

In [169]:
# 汽车整车
one_blk_corr = detail_industry_corr['880391']
one_blk_related_blk = one_blk_corr[one_blk_corr>0.75].sort_values(ascending= False)
汽车整车 = one_blk_related_blk.to_frame().join(detail_industry_code_name_map).merge(blk_category,left_on='名称',right_index=True)
汽车整车.insert(1,'index','880391')
汽车整车

,880391,index,名称,category
code,,,,
880391,1.000000,880391,汽车整车,大板块
880392,0.822812,880391,汽车配件,大板块


In [167]:
top1_stocks_non_small.loc['汽车整车']

代码                002594
名称                   比亚迪
ab_mv        5113.060000
blk_mv      14556.100000
mv_share       35.126579
count                 23
Name: 汽车整车, dtype: object

In [170]:
# top1_stocks_non_small.loc['汽车整车']
top1 = top1_stocks_non_small.loc['汽车整车']['代码']
top1_data = QA.QA_fetch_stock_day_adv(top1,'2022-01-01','2022-09-08').pivot('close').pct_change()
pd.concat([detail_industry_data.pivot('close')[['880391']].pct_change(),top1_data],axis=1).corr()

code,880391,002594
code,,
880391,1.000000,0.831608
002594,0.831608,1.000000


### 食品

In [173]:
# 食品
one_blk_corr = detail_industry_corr['880375']
one_blk_related_blk = one_blk_corr[one_blk_corr>0.70].sort_values(ascending= False)
食品 = one_blk_related_blk.to_frame().join(detail_industry_code_name_map).merge(blk_category,left_on='名称',right_index=True)
食品.insert(1,'index','880375')
食品

,880375,index,名称,category
code,,,,
880375,1.000000,880375,食品,大板块
880374,0.794839,880375,软饮料,小型板块
880373,0.792749,880375,乳制品,中小型板块
880383,0.769851,880375,红黄酒,小型板块


In [171]:
top1_stocks_non_small.loc['食品']

代码                603288
名称                  海天味业
ab_mv        3614.390000
blk_mv      14605.570000
mv_share       24.746655
count                 88
Name: 食品, dtype: object

In [174]:
# top1_stocks_non_small.loc['汽车整车']
top1 = top1_stocks_non_small.loc['食品']['代码']
top1_data = QA.QA_fetch_stock_day_adv(top1,'2022-01-01','2022-09-08').pivot('close').pct_change()
pd.concat([detail_industry_data.pivot('close')[['880375']].pct_change(),top1_data],axis=1).corr()

code,880375,603288
code,,
880375,1.000000,0.739262
603288,0.739262,1.000000


### 水力发电

In [51]:
# 水力发电
one_blk_corr = detail_industry_corr['880306']
one_blk_related_blk = one_blk_corr[one_blk_corr>0.60].sort_values(ascending= False)
水力发电 = one_blk_related_blk.to_frame().join(detail_industry_code_name_map).merge(blk_category,left_on='名称',right_index=True)
水力发电.insert(1,'index','880306')
水力发电

,880306,index,名称,category
code,,,,
880306,1.000000,880306,水力发电,中型板块
880307,0.676873,880306,火力发电,中型板块
880308,0.642668,880306,新型电力,中型板块
880466,0.629907,880306,路桥,中小型板块
880460,0.608738,880306,铁路,中小型板块
880468,0.608146,880306,港口,中小型板块
880471,0.602317,880306,银行,超大板块


In [176]:
top1_stocks_non_small.loc[['水力发电']]

,代码,名称,ab_mv,blk_mv,mv_share,count
细分行业,,,,,,
水力发电,600900,长江电力,5417.11,9594.84,56.458576,18


In [177]:
# top1_stocks_non_small.loc['汽车整车']
top1 = top1_stocks_non_small.loc['食品']['代码']
top1_data = QA.QA_fetch_stock_day_adv(top1,'2022-01-01','2022-09-08').pivot('close').pct_change()
pd.concat([detail_industry_data.pivot('close')[['880306']].pct_change(),top1_data],axis=1).corr()

code,880306,603288
code,,
880306,1.000000,0.179934
603288,0.179934,1.000000


## 申万子行业

In [22]:
child_industry = pd.read_csv('申万子行业.csv')
child_industry.sort_values('代码')

,代码,名称
54,880301,煤炭
15,880305,电力
17,880310,石油
44,880318,钢铁
43,880324,有色
22,880330,化纤
27,880335,化工
48,880344,建材
31,880350,造纸
55,880351,矿物制品


In [13]:
child_industry_code = child_industry['代码'].astype(str).to_list()
child_industry_code[:5]

['880431', '880490', '880494', '880492', '880418']

In [14]:
child_industry_data = QA.QA_fetch_index_day_adv(child_industry_code,'2022-01-01','2022-09-02')

In [37]:
child_industry_data.data.head()

open    close     high      low       vol        amount  \
date       code                                                                 
2022-01-04 880301   931.47   920.47   943.68   914.04  210057.0  1.968611e+10   
           880305  1938.98  1901.07  1947.99  1897.17  525302.0  3.565899e+10   
           880310   693.05   696.31   697.96   690.91  156730.0  7.833487e+09   
           880318  1105.75  1114.31  1118.64  1105.75  263546.0  1.546208e+10   
           880324  1048.57  1026.31  1049.79  1018.67  375823.0  6.011190e+10   

                   up_count  down_count    date_stamp    volume  
date       code                                                  
2022-01-04 880301        30           4  1.641226e+09  210057.0  
           880305        28          42  1.641226e+09  525302.0  
           880310        28           7  1.641226e+09  156730.0  
           880318        46           8  1.641226e+09  263546.0  
           880324        72          43  1.641226e+09  375823.0

In [51]:
# 
child_industry_pivot = child_industry_data.pivot('close')
child_industry_pivot.head()

code,880301,880305,880310,880318,880324,880330,880335,880344,880350,880351,...,880474,880476,880482,880489,880490,880491,880492,880493,880494,880497
date,,,,,,,,,,,,,,,,,,,,,
2022-01-04,920.47,1901.07,696.31,1114.31,1026.31,2784.80,2284.52,1789.53,1522.10,2289.38,...,1190.73,1337.87,1350.15,1664.03,1703.57,3399.73,3041.77,2789.40,2457.76,1114.81
2022-01-05,907.28,1846.79,699.07,1095.91,1001.69,2790.02,2234.56,1768.21,1499.92,2212.49,...,1183.91,1320.62,1354.78,1655.08,1674.04,3257.17,2933.95,2761.13,2483.45,1099.13
2022-01-06,907.51,1829.69,700.46,1123.51,1012.70,2858.15,2252.79,1800.38,1521.16,2210.11,...,1185.69,1365.43,1349.27,1653.45,1671.06,3241.99,2915.05,2725.85,2451.54,1111.73
2022-01-07,910.09,1803.95,726.81,1140.37,992.23,2828.08,2209.04,1822.83,1505.44,2155.91,...,1194.68,1380.33,1383.00,1618.84,1650.08,3192.75,2884.19,2672.64,2391.68,1102.10
2022-01-10,927.64,1795.26,727.75,1136.22,1000.28,2848.66,2221.10,1823.01,1514.43,2138.94,...,1198.05,1393.29,1400.01,1667.57,1663.25,3171.70,2899.51,2711.83,2396.91,1108.53


In [61]:
child_industry_corr = child_industry_pivot.pct_change().corr()

In [121]:
child_industry_corr.head()

code,880301,880305,880310,880318,880324,880330,880335,880344,880350,880351,...,880474,880476,880482,880489,880490,880491,880492,880493,880494,880497
code,,,,,,,,,,,,,,,,,,,,,
880301,1.000000,0.457005,0.672686,0.508798,0.479966,0.331723,0.384374,0.371876,0.281436,0.177597,...,0.379791,0.442363,0.418174,0.244306,0.169642,0.016649,0.148080,0.203849,0.231597,0.318186
880305,0.457005,1.000000,0.466193,0.705074,0.555050,0.473926,0.579693,0.677795,0.601600,0.447779,...,0.678953,0.659184,0.532161,0.500336,0.521441,0.338104,0.477805,0.506320,0.530488,0.597552
880310,0.672686,0.466193,1.000000,0.584882,0.502954,0.365056,0.412669,0.428355,0.397470,0.183294,...,0.515422,0.539637,0.465776,0.233717,0.225614,0.060220,0.196128,0.247566,0.299057,0.324592
880318,0.508798,0.705074,0.584882,1.000000,0.727472,0.620702,0.707699,0.823887,0.720836,0.553422,...,0.744609,0.763308,0.651465,0.606746,0.611808,0.404659,0.548855,0.622044,0.654952,0.683499
880324,0.479966,0.555050,0.502954,0.727472,1.000000,0.635683,0.889058,0.620745,0.671856,0.755497,...,0.583304,0.450880,0.359680,0.636690,0.683371,0.570895,0.702771,0.657808,0.598720,0.627190


In [103]:
child_industry.loc[:,('stock_code')] = child_industry['代码'].astype(str)
child_industry_code_name = child_industry.drop('代码',axis=1).set_index('stock_code')

In [105]:
# 电气设备
one_blk_corr = child_industry_corr[supper_big_industry_code[0]]
one_blk_related_blk = one_blk_corr[one_blk_corr>0.7].sort_values(ascending= False)
one_blk_related_blk.to_frame().join(child_industry_code_name)

,880446,名称
code,,
880446,1.000000,电气设备
880440,0.876329,工业机械
880351,0.843018,矿物制品
880492,0.814913,元器件
880437,0.809383,通用机械
880335,0.801402,化工
880390,0.787617,汽车类
880490,0.748017,通信设备
880448,0.735565,电器仪表


In [120]:
# 酿酒
one_blk_corr = child_industry_corr[supper_big_industry_code[1]]
one_blk_related_blk = one_blk_corr[one_blk_corr>0.6].sort_values(ascending= False)
one_blk_related_blk.to_frame().join(child_industry_code_name)

,880380,名称
code,,
880380,1.000000,酿酒
880372,0.699610,食品饮料
880387,0.650577,家用电器
880355,0.618630,日用化工


In [112]:
# 化工
one_blk_corr = child_industry_corr[supper_big_industry_code[2]]
one_blk_related_blk = one_blk_corr[one_blk_corr>0.7].sort_values(ascending= False)
one_blk_related_blk.to_frame().join(child_industry_code_name)

,880335,名称
code,,
880335,1.000000,化工
880324,0.889058,有色
880440,0.864302,工业机械
880351,0.817956,矿物制品
880456,0.805736,环境保护
880437,0.803928,通用机械
880446,0.801402,电气设备
880492,0.789784,元器件
880414,0.783309,商贸代理


In [115]:
# 医药
one_blk_corr = child_industry_corr[supper_big_industry_code[3]]
one_blk_related_blk = one_blk_corr[one_blk_corr>0.65].sort_values(ascending= False)
one_blk_related_blk.to_frame().join(child_industry_code_name)

,880400,名称
code,,
880400,1.000000,医药
880398,0.888376,医疗保健
880406,0.765079,商业连锁
880372,0.691861,食品饮料
880350,0.680243,造纸
880490,0.673468,通信设备
880335,0.672085,化工
880355,0.668702,日用化工
880494,0.667502,互联网


In [116]:
# 汽车类
one_blk_corr = child_industry_corr[supper_big_industry_code[4]]
one_blk_related_blk = one_blk_corr[one_blk_corr>0.65].sort_values(ascending= False)
one_blk_related_blk.to_frame().join(child_industry_code_name)

,880390,名称
code,,
880390,1.000000,汽车类
880440,0.837093,工业机械
880492,0.792099,元器件
880446,0.787617,电气设备
880437,0.785773,通用机械
880335,0.764853,化工
880351,0.762659,矿物制品
880490,0.753577,通信设备
880432,0.706670,运输设备


In [119]:
# 元器件
one_blk_corr = child_industry_corr[supper_big_industry_code[5]]
one_blk_related_blk = one_blk_corr[one_blk_corr>0.75].sort_values(ascending= False)
one_blk_related_blk.to_frame().join(child_industry_code_name)

,880492,名称
code,,
880492,1.000000,元器件
880490,0.917792,通信设备
880440,0.889661,工业机械
880491,0.861494,半导体
880448,0.847548,电器仪表
880493,0.820136,软件服务
880437,0.816570,通用机械
880446,0.814913,电气设备
880489,0.796969,IT设备


In [122]:
# 半导体
one_blk_corr = child_industry_corr['880491']
one_blk_related_blk = one_blk_corr[one_blk_corr>0.75].sort_values(ascending= False)
one_blk_related_blk.to_frame().join(child_industry_code_name)

,880491,名称
code,,
880491,1.000000,半导体
880492,0.861494,元器件
880490,0.800875,通信设备


In [123]:
# 热门板块关联分析
one_blk_corr = child_industry_corr['880302']
one_blk_related_blk = one_blk_corr[one_blk_corr>0.75].sort_values(ascending= False)
one_blk_related_blk.to_frame().join(child_industry_code_name)

KeyError: '880302'